# 1. Data Ingestion

In [1]:
import os

In [2]:
!pwd

/c/Users/lavanyaa/Projects/End-to-End-Project-ML-Flow/research


In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\lavanyaa\\Projects\\End-to-End-Project-ML-Flow'

In [19]:
#entity

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file:Path
    unzip_dir: Path


# #Constant - paths the yaml file

# from pathlib import Path

# CONFIG_FILE_PATH=Path("config/config.yaml")
# PARAMS_FILE_PATH=Path("params.yaml")
# SCHEMA_FILE_PATH = Path("schema.yaml")


In [20]:
# Configuration Manager 
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH,schema_filepath=SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config=self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(root_dir=config.root_dir, source_URL=config.source_URL, 
                                                    local_data_file=config.local_Data_file,
                                                    unzip_dir=config.unzip_dir)
        return data_ingestion_config

In [ ]:
# Component

import os
import urllib.request as request
import zipfile
from mlProject import logger
from mlProject.utils.common import get_size


class DataIngestion:
    def __init__(self,config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(url=self.config.source_URL, filename=self.config.local_data_file)
            logger.info(f"{filename} download with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        # print("ZIP File Path:", self.config.local_data_file)
        # print("File Size (bytes):", os.path.getsize(self.config.local_data_file))
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [22]:
#Pipeline

try:
    config=ConfigurationManager()
    data_ing_config=config.get_data_ingestion_config()
    data_ing=DataIngestion(config=data_ing_config)
    data_ing.download_file()
    data_ing.extract_zip_file()
except Exception as e:
    raise e

[2025-06-17 08:39:22,351:INFO:common: yaml file: config\config.yaml loaded successfully]
[2025-06-17 08:39:22,351:INFO:common: yaml file: params.yaml loaded successfully]
[2025-06-17 08:39:22,360:INFO:common: yaml file: schema.yaml loaded successfully]
[2025-06-17 08:39:22,360:INFO:common: created directory at: artifacts]
[2025-06-17 08:39:22,360:INFO:common: created directory at: artifacts/data_ingestion]
[2025-06-17 08:39:23,953:INFO:1184442725: artifacts/data_ingestion/data.zip download with following info: 
Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: B3EB:170B30:6BAC9:187273:6850DC9D
Accept-Ranges: bytes
Date: Tue, 17 Jun 2025 03:1

# Data validation

In [ ]:
# Entity
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)

class DataValidationConfig:
    root_dir: Path
    STATUS_FILE: str
    unzip_data_dir: Path
    all_schema: dict


In [ ]:
# Configuration Manager 
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH,schema_filepath=SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config=self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(root_dir=config.root_dir, source_URL=config.source_URL, 
                                                    local_data_file=config.local_Data_file,
                                                    unzip_dir=config.unzip_dir)
        return data_ingestion_config
    
    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.COLUMNS

        create_directories([config.root_dir])

        data_validation_config= DataValidationConfig(root_dir=config.unzip_data_dir,
                                                     rata_dir = config.5)